In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sklearn 
import numpy as np
%matplotlib inline

In [197]:
length=[] #short, medium, and long equal <60, between 60 and 120, and >120, respectively
games=pd.read_csv("gamesdataset.csv", encoding="latin1")
games=games[games.notnull()]
games.minplayers=games.minplayers.astype(str)
games.maxplayers=games.maxplayers.astype(str)
games["range"]=games.minplayers+"-"+games.maxplayers
games=games[games.playingtime.notnull()]
for i in games.playingtime:
    if i < 60 or i == 60:
        length.append("short")
    if i > 60 and i <=120:
        length.append("medium")
    if i >120:
        length.append("long")
    if i == np.nan:
        length.append(np.nan)
games["length"]=length  #turned as many things as possible into factors/categorical data


In [198]:
games.year=games.year.astype(int)

In [199]:
gametype=[]
for i in games.year:
    if i > 2000: 
        gametype.append("Euro")
    if i == 2000:
        gametype.append("Euro")
    if i < 2000:
        gametype.append("War")
games["gametype"]= gametype

In [200]:
plamount=[]
games["avgplayers"]=games.maxplayers.astype("float")+games.minplayers.astype("float")
for i in games.avgplayers:
    if i < 3 or i == 3:
        plamount.append("few") #1-2 person games
    if i > 3 and i < 9:
        plamount.append("average") #assuming a max of 3-5 players
    if i > 8:
        plamount.append("many")
games["playeramount"]=plamount
games.columns

Index(['id', 'description', 'image', 'maxplayers', 'maxplaytime', 'minage',
       'minplayers', 'minplaytime', 'name', 'playingtime', 'thumbnail', 'year',
       'boardgameartist', 'category', 'compilation', 'designer',
       'implementation', 'integration', 'mechanic', 'publisher', 'average',
       'weight', 'geekscore', 'median', 'numcomments', 'numweights', 'owned',
       'stddev', 'stats.subtype.boardgame.bayesaverage',
       'stats.subtype.boardgame.pos', 'trading', 'usersrated', 'wanting',
       'range', 'length', 'gametype', 'avgplayers', 'playeramount'],
      dtype='object')

In [222]:
games.category=games.category.astype(str)
games.publisher=games.publisher.astype(str)
games.mechanic=games.mechanic.astype(str)
cats=games.category.str.split(",", expand=True)
games["simplecats"]=cats[0]
mech=games.mechanic.str.split(",", expand=True)
games["simplemech"]=mech[0]

In [223]:
games.columns

Index(['id', 'description', 'image', 'maxplayers', 'maxplaytime', 'minage',
       'minplayers', 'minplaytime', 'name', 'playingtime', 'thumbnail', 'year',
       'boardgameartist', 'category', 'compilation', 'designer',
       'implementation', 'integration', 'mechanic', 'publisher', 'average',
       'weight', 'geekscore', 'median', 'numcomments', 'numweights', 'owned',
       'stddev', 'stats.subtype.boardgame.bayesaverage',
       'stats.subtype.boardgame.pos', 'trading', 'usersrated', 'wanting',
       'range', 'length', 'gametype', 'avgplayers', 'playeramount',
       'simplecats', 'simplemech'],
      dtype='object')

In [244]:
cols_to_transform=["length", "simplecats", "simplemech", "playeramount"]
ds=games.copy()
ds=pd.get_dummies(data=ds,columns=cols_to_transform)
ds=ds.drop(["id", "description", "maxplayers", "maxplaytime", "minage", "minplayers", "minplaytime", "name", "playingtime"], axis=1)
ds=ds.drop(["image", "thumbnail", "category", "compilation", "designer", "implementation", "integration", "mechanic"], axis=1)
ds=ds.drop(["year", "boardgameartist", "publisher"], axis=1)
ds=ds.drop(['stats.subtype.boardgame.bayesaverage', 'stats.subtype.boardgame.pos'], axis=1)
ds=ds.drop(["range", "avgplayers"], axis=1)

In [59]:
ratings=[]
games.columns
tdgs=games[["description","category", "mechanic", "publisher", "average", "geekscore"]]
tdgs=tdgs.dropna()
tdgs=tdgs[tdgs.geekscore>0]

In [60]:
tdgs.description=tdgs.description.str.replace("\d+", "")
y = tdgs.geekscore.mean()
for i in tdgs.geekscore:
    if i> y:
        ratings.append("Good")
    if i == y or i <y:
        ratings.append("Bad")
tdgs["gsratings"]=ratings
tdgs["docs"]=tdgs["description"]


In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
def make_xy(tdgs, vectorizer=None):
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(tdgs.docs)
    X=X.tocsc()
    y=(tdgs.gsratings=="Good").values.astype(np.int)
    return X,y
X,y=make_xy(tdgs)

In [51]:
from sklearn.model_selection import train_test_split
Xnb_train, Xnb_test, ynb_train, ynb_test = train_test_split(X, y, test_size=0.3, random_state=5)
clf=MultinomialNB()
clf.fit(Xnb_train, ynb_train)
print(clf.score(Xnb_train, ynb_train))
clf.score(Xnb_test, ynb_test)

0.892048046222


0.72163120567375882

In [52]:
from sklearn.model_selection import KFold
def cv_score(clf, X, y):
    result = 0.
    nfold = 5
    for train, test in KFold(nfold).split(X): # split data into train/test groups, 5 times
        clf.fit(X[train], y[train]) # fit the classifier, passed is as clf.
        result += clf.score(X[test], y[test]) # evaluate score function on held-out data
    return result / nfold # average

In [53]:
from sklearn.model_selection import train_test_split
_, itest = train_test_split(range(tdgs.shape[0]), train_size=0.7)
mask = np.zeros(tdgs.shape[0], dtype=np.bool)
mask[itest] = True

In [54]:
from sklearn.model_selection import cross_val_score
alphas = [.001,.01,.1, 1, 5, 10, 100]
best_min_df = [.001, .01,1,2,3,4,5] # YOUR TURN: put your value of min_df here.
scores_list=[]
#Find the best value for alpha and min_df, and the best classifier
results_list=[]
best_alpha = None
maxscore=-np.inf
for alpha in alphas:
    for df in best_min_df:
        vectorizer = CountVectorizer(min_df=df)
        Xthis, ythis = make_xy(tdgs, vectorizer)
        Xtrainthis = Xthis[mask]
        ytrainthis = ythis[mask]
        clf=MultinomialNB(alpha=alpha)
        score=cv_score(clf,Xtrainthis, ytrainthis)
        scores_list.append((score,alpha, df))
for i in scores_list:
    if i[0] > maxscore:
        maxscore=i[0]
        results_list.append(i)
x=len(results_list)-1
best_alpha=results_list[x][1]
best_min_df=results_list[x][2]
results_list

[(0.69893617021276611, 0.001, 0.001),
 (0.7088652482269503, 0.001, 0.01),
 (0.73546099290780143, 1, 1),
 (0.7365248226950355, 5, 0.001),
 (0.73900709219858152, 5, 5)]

In [55]:
vectorizer = CountVectorizer(min_df=best_min_df, stop_words="english")
X, y = make_xy(tdgs, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

Accuracy on training data: 0.845390
Accuracy on test data:     0.740611


In [56]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest, clf.predict(xtest)))

[[4503  318]
 [1388  368]]


In [57]:
words = np.array(vectorizer.get_feature_names())

x = np.eye(xtest.shape[1])
probs = clf.predict_log_proba(x)[:, 0]
ind = np.argsort(probs)

good_words = words[ind[:50]]
bad_words = words[ind[-50:]]

good_prob = probs[ind[:50]]
bad_prob = probs[ind[-50:]]

print("Good words\t     P(good | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))
    
print("Bad words\t     P(good | word)")
for w, p in zip(bad_words, bad_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

Good words	     P(good | word)
            alhambra 0.74
              danish 0.73
              swedes 0.73
       civilizations 0.70
             cowboys 0.69
           simonitch 0.69
             rangers 0.66
            manstein 0.66
          protestant 0.65
                 max 0.65
              worker 0.65
            chariots 0.64
            ardennes 0.64
            district 0.64
              cattle 0.64
               beads 0.63
              martin 0.63
                 par 0.63
               omega 0.63
               epoch 0.63
               adolf 0.63
              cowboy 0.62
                  pi 0.61
            proposal 0.61
                huts 0.61
               raquo 0.61
            baseball 0.61
              blocks 0.60
               bilbo 0.60
             swedish 0.59
                 gmt 0.59
               roses 0.59
          settlement 0.58
               oases 0.58
           vegetable 0.58
             eclipse 0.58
         plantations 0.58
       